In [1]:
import sys 
sys.path.append("..")

import pandas as pd
import numpy as np
import torch
from DIN.data import get_amazon_data_dict  # 针对性修改

from Tools.models.ranking import DIN
from Tools.trainers import CTRTrainer
from Tools.basic.features import DenseFeature, SparseFeature, SequenceFeature
from Tools.utils.data import DataGenerator, generate_seq_feature, df_to_dict, pad_sequences

dataset_path = '../data/amazon_electronic_datasets.csv'
model_name='din'
epoch = 10
learning_rate = 1e-3
batch_size=2048
weight_decay=1e-3
save_dir='./'
seed=2023
# device='cpu'
device = torch.device("cuda")

/data0/ygq/anaconda3/envs/pytorch/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(seed)
features, target_features, history_features, (train_x, train_y), (val_x, val_y), (test_x, test_y) = get_amazon_data_dict(dataset_path)

dg = DataGenerator(train_x, train_y)
train_dataloader, val_dataloader, test_dataloader = dg.generate_dataloader(x_val=val_x, y_val=val_y, x_test=test_x, y_test=test_y, batch_size=batch_size)

========== Start Amazon ==========


generate sequence features: 100%|██████████████████████████████████████████| 192403/192403 [00:28<00:00, 6666.75it/s]


========== Generate input dict ==========


In [3]:
model = DIN(features=features, history_features=history_features, target_features=target_features, mlp_params={"dims": [256, 128]}, attention_mlp_params={"dims": [256, 128]})

In [4]:
ctr_trainer = CTRTrainer(model, optimizer_params={"lr": learning_rate, "weight_decay": weight_decay}, n_epoch=epoch, earlystop_patience=4, device=device, model_path=save_dir)
ctr_trainer.fit(train_dataloader, val_dataloader)
auc = ctr_trainer.evaluate(ctr_trainer.model, test_dataloader)
print(f'test auc: {auc}')

epoch: 0


validation: 100%|██████████████████████████████████████████████████████████████████| 187/187 [00:03<00:00, 50.02it/s]


epoch: 0 validation: auc: 0.828903882307786
epoch: 1


validation: 100%|██████████████████████████████████████████████████████████████████| 187/187 [00:03<00:00, 50.91it/s]


epoch: 1 validation: auc: 0.8469871425097393
epoch: 2


...

epoch: 6 validation: auc: 0.8466337637907028
epoch: 7


validation: 100%|██████████████████████████████████████████████████████████████████| 187/187 [00:03<00:00, 50.31it/s]


epoch: 7 validation: auc: 0.8464408795741024
validation: best auc: 0.8516600810587673


validation: 100%|██████████████████████████████████████████████████████████████████| 187/187 [00:03<00:00, 50.72it/s]

test auc: 0.8461158754627309
